# Hybrid digital twin of a Li-ion battery


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

### 1. Load experimental data

In [98]:
import pandas as pd
df = pd.read_csv('2019.csv')

In [99]:
import plotly.express as px
fig = px.scatter_matrix(df.drop(columns=['Time', 'month', 'wind_dir',
                                        'mslp']), 
                                )
fig.update_traces(marker=dict(size=2,
                              color='crimson',
                              symbol='square')),
fig.update_traces(diagonal_visible=False)
fig.update_layout(
    title='Climate dataser',
    width=900,
    height=1200,
)
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                   'paper_bgcolor': 'white',}, 
                    template='plotly_white',
                    font=dict(size=7)
                    )

fig.show()

In [100]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Time'], 
                         y=df['tluft'],
                         mode='lines',
                         name='Environmental Temerature',
                         marker_size=3, 
                         line=dict(color='crimson', width=1)    
                        ))
fig.update_layout(
                  title="Temperature during January-March 2019",
                  xaxis_title="Date Time",
                  yaxis_title="Temperature (C)"
    )
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                   'paper_bgcolor': 'white',}, 
                    template='plotly_white')

### 2. Define the physical model of building

Physical model of the building based on base_model.pdf is simulated in Revit2021. envelope_properties.csv consists of the walls, floors and ceiling of the building as the output of the Revit model. The main factor affcting heating load regarding the materials and componets of a building is envelope area and the ratio of envelope/fenestration.<br>

in this section the type of envelopes and their ratio has shown as the graph.<br>


In [4]:
import plotly.express as px

data = px.data.gapminder()
df_building = pd.read_csv('envelopes_properties.csv')
data_building = data[df_building]
fig = px.bar(data_building, x=df_building['Envelope Type'], y=df_building['Envelop Area'],
             hover_data=[df_building['Fenestration Area']], color=df_building['Fenestration Area'],
              height=600)
fig.update_layout(
                  title="Envelopes Area",
                  xaxis_title="Envelope Name",
                  yaxis_title="Area (m2)"
    )
fig.show()

In [5]:
import plotly.express as px

data = px.data.gapminder()
df_building = pd.read_csv('envelopes_properties.csv')
data_building = data[df_building]
fig = px.bar(data_building, x=df_building['Envelope Type'], y=df_building['Fenestration Area'],
             hover_data=[df_building['ratio']], color=df_building['ratio'],
              height=600)
fig.update_layout(
                  title="Ratio of Envelopes to Fenestration",
                  xaxis_title="Envelope Name",
                  yaxis_title="Fenestration Area (m2)"
    )
fig.show()

In [6]:
import plotly.express as px

data = px.data.gapminder()
df_building = pd.read_csv('envelopes_properties.csv')
data_building = data[df_building]
fig = px.bar(data_building, x=df_building['Envelope Type'], y=df_building['u factor'],
             hover_data=[df_building['Material']], color=df_building['Material'],
              height=600)
fig.update_layout(
                  title="U-factor of Envelopes",
                  xaxis_title="Envelope Name",
                  yaxis_title="U (W/m2k)"
    )
fig.show()

### 3. Define the heating load model of the building

Heating loads are due to heat losses by:


1) Conductive heat loss through envelope (walls, windows, doors, floors, roof, etc.).<br>

2) Infiltration, exfiltration and ventilation air.<br>


Physical model of conductive heating load according [1]. The basic equation is:<br><br>

<center> $Q_{Conductive} = UA\Delta t$ </center><br>

Where $Q_{Conductive}$ is the conductive heating load and $\Delta t$ is the difference between indoor and outdoor design air temperature (K). $U$ the overall heat transfer coefficient $(W/m^2 K)$ where: <br><br>
<center> $U = \frac 1 {R_{total}}$ </center><br> 

where $R_{total}$ is heating resistancee.<br> 

Infilteration and ventilation load are devided to:<br>

1) Sensible Heating Load<br>

2) Latent Heating Load<br>


The basic equation for the physical model of sensible heating load could be written as follows:<br><br>


<center> $Q_{Sencible} = {CMH}×\Delta t$ </center><br>

The basic equation for the physical model of latent heating load could be written as follows:<br><br>


<center> $Q_{Latent} ={CMH}×\Delta r$ </center><br>

where ${CMH}$ is airflow $(m^3/h)$ .<br>

$\Delta t$ is the difference between indoor and outdoor air temperature (K).<br>

$\Delta t$ is the difference between indoor and outdoor air humidity $\frac {kg} {kg_{da}}$.<br><br>

$CMH$ for infiltration could be written as follows:<br><br>

<center> $CMH_i = {A_l} {IDF}$ </center><br><br>

Where ${A_l}$ is the building effective leakage area:<br><br>

<center> ${A_l} = {A_{es}} {A_{ul}}$ </center><br><br>

 ${A_{es}}$ is building exposed surface area $(m^2)$ <br>
 ${A_{ul}}$ is unit leakage area, cm2/m2 (from Table 3) $(\frac{cm^2}{m^2})$ <br><br> 
 
 $CMH$ for ventilation could be written as follows:<br><br>

<center> $CMH_v = 0.15{A_{cf}} + 3.5 ({N_{br}} +1)$ </center><br><br>

Where ${A_{cf}}$ is = building conditioned floor area and ${N_{br}}$ is the number of bedrooms (not less than 1).<br><br>



- [1] *ASHRAE (American Society of Heating, Refrigerating and Air-Conditioning Engineers
) 2021.* 

In [96]:
from math import e
df_design = pd.read_csv('building_design_construction_properties.csv')

""" Conductive load calculation"""
q_conductive = (df_building['Envelop Area']-df_building['Fenestration Area']).values *(df_building['u factor'].values)*(df_design['heating  design temperature(K)'].iloc[0:1].values[0])
q_conductive_total = sum(abs(q_conductive))
total_envelopes_area =sum(df_building ['Envelop Area'])

"""infiltration and airflow calculation"""
#according to the building_design_construction_properties.csv and tables from ASHRAE:
idf = 0.06
aul = 2.8
nbr = df_design['Number of Bedrooms'].iloc[0:1].values[0]

def floors_area(df_building):
    if df_building['Group'] == 1:
        return df_building['Envelop Area']
    else:
        return 0      

a_floor = df_building.apply(floors_area, axis=1)
acf = sum(a_floor.values)
scf = df_building ['Envelop Area']
cmh_flow_infiltration= (total_envelopes_area * idf * aul)
cmh_flow_ventilation= ((0.15*acf) + (3.5*(nbr+1)))

""" defference of temperature inside and outside"""
inside_temp_winter = df_design['Average inside (k) heating'].iloc[0:1].values[0]
outside_temperature = df['tluft'].values
T = outside_temperature
dt_temperature = abs(inside_temp_winter-outside_temperature)

""" Converting relative humidity to absolute hu,idity"""
inside_humidity_winter = df_design['Moisture difference heating (KG/KG DA)'].iloc[0:1].values[0]
outside_humidity_relative = df['rh'].values
press = df['mslp'].values
e_vapour_pressure_water = 6.107799961 + T*(0.4436518521+T*(0.01428945805+T*(0.0002650648471+T*(0.000003031240396+T*(0.00000002034080948+T*(0.00000000006136820929))))))
p_h20 = (outside_humidity_relative * e_vapour_pressure_water)/100
x_h20 = p_h20 / press
abs_humidity_gr = (x_h20 * 18.01534) /((x_h20 * 18.01534)+((1-18.01534)*28.9644))

""" infiltration load"""
q_infiltration_sensible = cmh_flow_infiltration* (abs(abs_humidity_gr)) 
q_infiltration_lateral =  cmh_flow_infiltration * dt_temperature  

q_infiltration_total = q_infiltration_sensible + q_infiltration_lateral


"""ventilation load"""
q_ventilation_sensible =   cmh_flow_ventilation * (abs(abs_humidity_gr)) 
q_ventilation_lateral =  cmh_flow_ventilation * dt_temperature 

q_ventilation_total = q_ventilation_sensible + q_ventilation_lateral

"""Total Heating load"""

heatin_load_total = q_conductive_total + q_infiltration_total + q_ventilation_total










In [97]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Time'], 
                         y= heatin_load_total,
                         mode='lines',
                         name='Environmental Temerature',
                         marker_size=3, 
                         line=dict(color='crimson', width=1)    
                        ))
fig.update_layout(
                  title="Heating Load (W) during January-March 2021",
                  xaxis_title="Date Time",
                  yaxis_title="Heating Load (W)"
    )
fig.update_layout({'plot_bgcolor': '#f2f8fd',
                   'paper_bgcolor': 'white',}, 
                    template='plotly_white')